In [1]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax
import re

Data Load

In [2]:
df_original = pd.read_excel('../data/raw/tweets_12-27Mayo.xlsx')  
df_post = df_original['POST']
print("The dataframe has {} rows and {} columns.".format(df_original.shape[0], df_original.shape[1]))

The dataframe has 776 rows and 19 columns.


Load a pretrained model and its corresponding tokenizer. The model is cardiffnlp/twitter-roberta-base-sentiment, which is a model trained for sentiment analysis.

In [3]:
MODEL = "cardiffnlp/xlm-twitter-politics-sentiment"
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
tokenizer = AutoTokenizer.from_pretrained(MODEL)

Define the sentiment labels: Negative, Neutral, and Positive.

In [4]:
labels = ['Negative', 'Neutral', 'Positive']


The data frame df is defined to be df_post, which presumably contains the data you're interested in processing.

In [5]:
df = df_post

Define a function preprocess_tweet to preprocess tweets. This function replaces @mentions with "@user" and links (starting with http) with "http".

In [6]:
def preprocess_tweet(tweet):
    if isinstance(tweet, str):
        tweet = tweet.lower()  # Convert to lowercase
      
        tweet_words = []
        for word in tweet.split(' '):
            if word.startswith('@') and len(word) > 1:
                word = '@user'
            elif word.startswith('http'):
                word = "http"
            tweet_words.append(word)

        return " ".join(tweet_words)
    else:
        return ""

def preprocess_tweet(tweet):
    tweet_words = []
    for word in tweet.split(' '):
        if word.startswith('@') and len(word) > 1:
            word = '@user'
        elif word.startswith('http'):
            word = "http"
        tweet_words.append(word)
    return " ".join(tweet_words)


def preprocess_tweet(tweet):
    if isinstance(tweet, float):
        return ''
    
    tweet_words = []
    for word in str(tweet).split(' '):
        if word.startswith('@') and len(word) > 1:
            word = '@user'
        elif word.startswith('http') or word.startswith('www'):
            word = '@url'
        tweet_words.append(word)
    
    return ' '.join(tweet_words)

Apply the preprocess_tweet function to all tweets in the dataframe and store the results in a new column named processed_text.

In [7]:
df['processed_text'] = df.apply(preprocess_tweet)

C:\Users\Pablo\AppData\Local\Temp\ipykernel_15092\139711792.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['processed_text'] = df.apply(preprocess_tweet)


Define a function analyze_sentiment to analyze the sentiment of tweets using the loaded model and tokenizer. This function returns a dictionary of sentiment labels and their corresponding scores.

In [8]:
def analyze_sentiment(tweet):
    encoded_tweet = tokenizer(tweet, return_tensors='pt')
    output = model(**encoded_tweet)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    return dict(zip(labels, scores))


Apply the analyze_sentiment function to all processed tweets in the dataframe and store the results in a new column named sentiment_analysis.

In [9]:
df['sentiment_analysis'] = df['processed_text'].apply(analyze_sentiment)


Break down the sentiment_analysis column into separate columns for each sentiment label.

In [10]:
sentiment_df = df['sentiment_analysis'].apply(pd.Series)


Concatenate the original data frame with the new sentiment dataframe along columns (axis=1).

In [11]:
df = pd.concat([df_original, sentiment_df], axis=1)


Determine the final sentiment for each tweet by choosing the sentiment label with the highest score, and store this in a new column named final_sentiment.

In [12]:
df['final_sentiment'] = sentiment_df.idxmax(axis=1)


In [13]:
df.head()

,Nº,PARTIDO,CANDIDATO,NICK,FOLLOWERS,FECHA,POST,VIDEO,FOTO,REPOST,...,SHARED,LIKES,VIEWED,VOTOS,PORCENTAJE,ESCAÑOS,Negative,Neutral,Positive,final_sentiment
0,1,PP,Isabel Díaz Ayuso,@IdiazAyuso,912100,2023-05-12,"Comenzamos la campaña, una vez más, junto a la...",SI,NO,NO,...,261,1260,58700.0,1586985,0.4734,71,0.034475,0.037882,0.927643,Positive
1,2,PP,Isabel Díaz Ayuso,@IdiazAyuso,912100,2023-05-12,"Madrid es la región del Espíritu de Ermua, la ...",NO,NO,NO,...,561,2127,101600.0,1586985,0.4734,71,0.904359,0.060345,0.035295,Negative
2,3,PP,Isabel Díaz Ayuso,@IdiazAyuso,912100,2023-05-12,"Majadahonda con ganas de Libertad, familia, un...",NO,SI,NO,...,213,1042,59700.0,1586985,0.4734,71,0.014554,0.021517,0.963929,Positive
3,4,PP,Isabel Díaz Ayuso,@IdiazAyuso,912100,2023-05-13,❤️❤️,NO,SI,SI,...,343,2958,159100.0,1586985,0.4734,71,0.005584,0.010402,0.984014,Positive
4,5,PP,Isabel Díaz Ayuso,@IdiazAyuso,912100,2023-05-13,Presidente: líbranos del mal.,NO,SI,NO,...,549,2592,330800.0,1586985,0.4734,71,0.680318,0.128941,0.190741,Negative


We export it to csv.

In [16]:
df.to_csv('../data/processed/df_Sentiment_pred_roberta_politic.csv')